# Project 3 - Code for neo4j business case for AGM

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

# Business Case for Neo4j - Proposed Solution

* Build a graph database in Neo4j for the BART system
* Use the Louvain Modularity Algorithm to identify strong knit communities of stations (11 communities identified)
* From each community, pick one station that has the potential to reach the maximum customer base (Using geodesic population grids)
* Use the selected 11 stations as pilot delivery points for phase I

# Included modules

In [38]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

pd.set_option('display.max_rows', 1000)

# Supporting code

Code reused from the labs and project 3 exercise

In [39]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [40]:
session = driver.session(database="neo4j")

In [41]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [42]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

cursor = connection.cursor()

In [43]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [44]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    #print("\n-------------------------------------------------------------------------------")
    #print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    #print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        #print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    #print("\n-------------------------------------------------------------------------------")
    #print("  Total Population: ", f'{total_population:10,}')
    #print("-------------------------------------------------------------------------------")
    #
    return total_population

In [45]:
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

# Louvain Modularity
We use the Louvaine Modularity community detction algorithm to group stations into communities. The premise is that if there are mutiple sttaions in a close knit community we whave good chances of reaching out to the population served by all these sttaions by just having a delivery point at one of the staaions. This helps us optimize our delivery infrastructure while maximizing return on investment.

In [46]:
# Run the Louvain Modularity algorithm on the graph created in the exercise 
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)
query = """

CALL gds.louvain.stream('ds_graph')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

communities_df = my_neo4j_run_query_pandas(query)

print(communities_df)
    
 

                            name  community intermediate_community
0     arrive 16th Street Mission          7                   None
1     arrive 24th Street Mission          7                   None
2       blue 16th Street Mission          7                   None
3       blue 24th Street Mission          7                   None
4     depart 16th Street Mission          7                   None
5     depart 24th Street Mission          7                   None
6      green 16th Street Mission          7                   None
7      green 24th Street Mission          7                   None
8        red 16th Street Mission          7                   None
9        red 24th Street Mission          7                   None
10    yellow 16th Street Mission          7                   None
11    yellow 24th Street Mission          7                   None
12           arrive Civic Center         21                   None
13          arrive Powell Street         21                   

# Data Clean up
The Louvain algorithm has identified 11 communities. We need to clean this data as each station has multiple nodes assoacited with it for arrival and departure. We need to extract just the station names and corresponding community number.

In [47]:
communities_df['Station'] = communities_df['name'].str.split().str[1:].str.join(' ')
#Drop the duplicate station names
communities_df.drop_duplicates('Station', inplace=True)
#Now that the sttaion names have bene extracted from the original node names, remove the unrequired columns
communities_df.drop(columns=['name', 'intermediate_community'], inplace=True)
print(communities_df)

     community               Station
0            7   16th Street Mission
1            7   24th Street Mission
12          21          Civic Center
13          21         Powell Street
24          29           Balboa Park
25          29             Daly City
26          29             Glen Park
42          31                 Ashby
43          31     Downtown Berkeley
44          31      El Cerrito Plaza
45          31  El Cerrito del Norte
46          31        North Berkeley
47          31              Richmond
66          47              Bay Fair
67          47         Castro Valley
68          47                Dublin
69          47               Hayward
70          47           San Leandro
71          47         South Hayward
72          47           West Dublin
93          51              Coliseum
94          51             Fruitvale
95          51          Lake Merritt
96          51                   OAK
112         59           Embarcadero
113         59     Montgomery Street
1

# Population 
Get the puplutaon for each stattion and add it to the dataframe 

In [50]:
communities_df['Population'] = communities_df.apply(lambda x: my_station_get_zips(x['Station'], 5), axis=1)
communities_df.rename(columns={'community': 'Community'}, inplace=True)
print(communities_df)

     Community               Station Population
0            7   16th Street Mission     870044
1            7   24th Street Mission     989138
12          21          Civic Center     870044
13          21         Powell Street     870044
24          29           Balboa Park     936912
25          29             Daly City     805122
26          29             Glen Park     986074
42          31                 Ashby     487206
43          31     Downtown Berkeley     448042
44          31      El Cerrito Plaza     355903
45          31  El Cerrito del Norte     369372
46          31        North Berkeley     445973
47          31              Richmond     301909
66          47              Bay Fair     457901
67          47         Castro Valley     403433
68          47                Dublin     213807
69          47               Hayward     467447
70          47           San Leandro     494839
71          47         South Hayward     411490
72          47           West Dublin    

# Stations with max population within each community

In [25]:
idx = communities_df.groupby(['Community'])['Population'].transform(max) == communities_df['Population']
communities_df[idx].drop_duplicates('Community')

,community,Station,Population
1,7,24th Street Mission,989138
12,21,Civic Center,870044
26,29,Glen Park,986074
42,31,Ashby,487206
70,47,San Leandro,494839
94,51,Fruitvale,544423
113,59,Montgomery Street,826249
138,63,Rockridge,536232
161,67,19th Street,576172
175,85,Colma,597747
